In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
import pandas
import json
import requests
from time import sleep
from tqdm.notebook import tqdm, trange

In [64]:
def update_names():
    op = webdriver.ChromeOptions()
    op.add_argument("--window-size=1920,1080")
    op.add_argument("--start-maximized")
    op.add_argument("--disable-gpu")
    op.add_argument('--disable-extensions')
    op.add_argument('--no-sandbox')
    op.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(), options=op)

    url_games = 'https://www.boardgamers.space/boardgame/gaia-project/games'

    games_set = set()

    with open("data/game_names.txt", "r") as names_file:
        lines = names_file.readlines()
        for line in lines:
            games_set.add(line.rstrip('\n'))
            
    with open("data/game_names.txt", "w") as names_file:
        for n in games_set:
            names_file.write(n + '\n')

    names_file.close()

    driver.get(url=url_games)
    finished_button = driver.find_element_by_partial_link_text("Finished")
    finished_button.click()
    
    sleep(.5)
    while driver.execute_script("return document.readyState") != "complete":
            sleep(.5)
    
    num_games_found = int(driver.find_element_by_class_name("card-title").text.split('(')[1].split(')')[0])

    # link to click for next page
    next_button = driver.find_elements_by_class_name("page-link")[-2]
    # parent <li> can be disabled if last page
    next_item = driver.find_elements_by_class_name("page-item")[-2]
    # list of disabled items
    disabled_items = driver.find_elements_by_class_name("disabled")

    # list of names items
    game_names = driver.find_elements_by_class_name("game-name")

    with open("data/game_names.txt", "a") as names_file:

        for name in game_names:
            n = name.text
            if n not in games_set:
                names_file.write(n + '\n')

        for i in range((num_games_found - len(games_set)) // 10):
            try:
                next_button.click()
                sleep(.5)

                # wait for JS to load -> elements are same, only text and class are updated
                while driver.execute_script("return document.readyState") != "complete":
                    sleep(.5)

                next_button = driver.find_elements_by_class_name("page-link")[-2]
                next_item = driver.find_elements_by_class_name("page-item")[-2]
                disabled_items = driver.find_elements_by_class_name("disabled")

                game_names = driver.find_elements_by_class_name("game-name")

                for name in game_names:
                    n = name.text
                    if n not in games_set:
                        names_file.write(n + '\n')
            except:
                break

    names_file.close()
    driver.close()

In [65]:
#driver.close()
update_names()



====== WebDriver manager ======
Current chromium version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 chromium
Driver [/home/tim/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache
/tmp/ipykernel_270122/2743936846.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(), options=op)
/tmp/ipykernel_270122/2743936846.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  finished_button = driver.find_element_by_partial_link_text("Finished")
/tmp/ipykernel_270122/2743936846.py:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  num_games_found = int(driver.find_element_by_class_name("card-title").text.split('(')[1].split(')')[0])
/tmp/ipykernel_270122/2743936846.py:37: DeprecationWarning: find_elements_by_* commands are de

In [66]:
games_set = set()

with open("data/game_names.txt", "r") as names_file:
    lines = names_file.readlines()
    for line in lines:
        games_set.add(line.rstrip('\n'))

In [67]:
len(games_set)

32609

In [57]:
"""
Combination of below cells to update the game names file

Currently set to run through whole set and checked for missed
"""

#link to click for next page
next_button = driver.find_elements_by_class_name("page-link")[-2]
#parent <li> can be disabled if last page
next_item = driver.find_elements_by_class_name("page-item")[-2]
#list of disabled items
disabled_items = driver.find_elements_by_class_name("disabled")

#list of names items
game_names = driver.find_elements_by_class_name("game-name")

games_set = set()

pbar = tqdm(total=32760)

with open("data/game_names.txt", "r") as names_file:
    lines = names_file.readlines()
    for line in lines:
        games_set.add(line)

names_file.close()
    
with open("data/game_names.txt", "a") as names_file:
    
    for name in game_names:
        n = name.text
        if n not in games_set:
            names_file.write(n + '\n' )
    
    while next_item not in disabled_items:
        try:
            next_button.click()
            sleep(.1)

            #wait for JS to load -> elements are same, only text and class are updated
            while driver.execute_script("return document.readyState") != "complete":
                sleep(.05)

            next_button = driver.find_elements_by_class_name("page-link")[-2]
            next_item = driver.find_elements_by_class_name("page-item")[-2]
            disabled_items = driver.find_elements_by_class_name("disabled")

            game_names = driver.find_elements_by_class_name("game-name")

            for name in game_names:
                n = name.text
                if n not in games_set:
                    names_file.write(n + '\n' )
                pbar.update(1)
        except Exception as e:
            print(e)
            break

names_file.close()

/tmp/ipykernel_270122/2452491508.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_button = driver.find_elements_by_class_name("page-link")[-2]
/tmp/ipykernel_270122/2452491508.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_item = driver.find_elements_by_class_name("page-item")[-2]
/tmp/ipykernel_270122/2452491508.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  disabled_items = driver.find_elements_by_class_name("disabled")
/tmp/ipykernel_270122/2452491508.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  game_names = driver.find_elements_by_class_name("game-name")


  0%|          | 0/32760 [00:00<?, ?it/s]

/tmp/ipykernel_270122/2452491508.py:44: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_button = driver.find_elements_by_class_name("page-link")[-2]
/tmp/ipykernel_270122/2452491508.py:45: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_item = driver.find_elements_by_class_name("page-item")[-2]
/tmp/ipykernel_270122/2452491508.py:46: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  disabled_items = driver.find_elements_by_class_name("disabled")
/tmp/ipykernel_270122/2452491508.py:48: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  game_names = driver.find_elements_by_class_name("game-name")


In [124]:
driver.close()